<a href="https://colab.research.google.com/github/racoci/whisper_api/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from IPython.display import Javascript, Audio
from io import BytesIO
from base64 import b64decode
from google.colab import output

display(Javascript("""
    const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
    const b2text = blob => new Promise(resolve => {
      const reader = new FileReader()
      reader.onloadend = e => resolve(e.srcElement.result)
      reader.readAsDataURL(blob)
    })
    var record = time => new Promise(async resolve => {
      stream = await navigator.mediaDevices.getUserMedia({ audio: true })
      recorder = new MediaRecorder(stream)
      chunks = []
      recorder.ondataavailable = e => chunks.push(e.data)
      recorder.start()
      await sleep(time)
      recorder.onstop = async ()=>{
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
      }
      recorder.stop()
    })
"""))
s=output.eval_js(f"record({5000})")
b = b64decode(s.split(',')[1])
audio_file = BytesIO(b)

<IPython.core.display.Javascript object>

In [19]:
!pip install git+https://github.com/racoci/whisper_api

  Cloning https://github.com/racoci/whisper_api to /tmp/pip-req-build-scx7cija
  Running command git clone --filter=blob:none --quiet https://github.com/racoci/whisper_api /tmp/pip-req-build-scx7cija
  Resolved https://github.com/racoci/whisper_api to commit 67da02ce4494800bcf1a123f084eb2b1e217928b
  Preparing metadata (setup.py) ... done
  Using cached openai_whisper-20231117-py3-none-any.whl


In [1]:
!git clone https://github.com/racoci/whisper_api

fatal: destination path 'whisper_api' already exists and is not an empty directory.


usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd --help

error: no commands supplied


In [17]:
%cd whisper_api
!python test_api.py

/content/whisper_api
Watching for file changes with StatReloader
Performing system checks...

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 496, in _make_request
    conn.request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", 

python3: can't open file '/content/test_api.py': [Errno 2] No such file or directory


In [20]:
import subprocess
# Start the Django server in a separate process
server_process = subprocess.Popen(['python', 'whisper_api/manage.py', 'runserver'])

In [21]:
import os
import requests
import time

# Set the API endpoint URL
url = 'http://localhost:8000/api/audio_to_text/'


# Prepare the file for upload
files = {'audio_file': audio_file}

try:
    # Send a POST request to the API endpoint with the audio file
    response = requests.post(url, files=files)

    # Check the response status code
    if response.status_code == 200:
        # Print the transcription result
        result = response.json()
        print('Transcription:', result['text'])
        print('Confidence:', result)
    else:
        print('Error:', response.status_code)
finally:
    # Terminate the Django server process
    server_process.terminate()

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/audio_to_text/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7adb3be567a0>: Failed to establish a new connection: [Errno 111] Connection refused'))